In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d uciml/breast-cancer-wisconsin-data
!unzip breast-cancer-wisconsin-data.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data
License(s): CC-BY-NC-SA-4.0
  0% 0.00/48.6k [00:00<?, ?B/s]
100% 48.6k/48.6k [00:00<00:00, 50.4MB/s]
Archive:  breast-cancer-wisconsin-data.zip
  inflating: data.csv                


In [ ]:
# Fuzzify data points based on custom rules
def fuzzify_radius_mean(value):
    if value < 12:
        return {"small": 1.0, "medium": 0.0, "large": 0.0}
    elif 12 <= value <= 18:
        return {"small": (18 - value) / 6, "medium": (value - 12) / 6, "large": 0.0}
    elif 18 < value <= 25:
        return {"small": 0.0, "medium": (25 - value) / 7, "large": (value - 18) / 7}
    else:
        return {"small": 0.0, "medium": 0.0, "large": 1.0}

def fuzzify_texture_mean(value):
    if value < 10:
        return {"soft": 1.0, "medium": 0.0, "hard": 0.0}
    elif 10 <= value <= 20:
        return {"soft": (20 - value) / 10, "medium": (value - 10) / 10, "hard": 0.0}
    elif 20 < value <= 30:
        return {"soft": 0.0, "medium": (30 - value) / 10, "hard": (value - 20) / 10}
    else:
        return {"soft": 0.0, "medium": 0.0, "hard": 1.0}

def fuzzify_perimeter_mean(value):
    if value < 50:
        return {"small": 1.0, "medium": 0.0, "large": 0.0}
    elif 50 <= value <= 100:
        return {"small": (100 - value) / 50, "medium": (value - 50) / 50, "large": 0.0}
    elif 100 < value <= 150:
        return {"small": 0.0, "medium": (150 - value) / 50, "large": (value - 100) / 50}
    else:
        return {"small": 0.0, "medium": 0.0, "large": 1.0}

def fuzzify_area_mean(value):
    if value < 400:
        return {"small": 1.0, "medium": 0.0, "large": 0.0}
    elif 400 <= value <= 700:
        return {"small": (700 - value) / 300, "medium": (value - 400) / 300, "large": 0.0}
    elif 700 < value <= 1000:
        return {"small": 0.0, "medium": (1000 - value) / 300, "large": (value - 700) / 300}
    else:
        return {"small": 0.0, "medium": 0.0, "large": 1.0}

def fuzzify_smoothness_mean(value):
    if value < 0.1:
        return {"soft": 1.0, "medium": 0.0, "hard": 0.0}
    elif 0.1 <= value <= 0.2:
        return {"soft": (0.2 - value) / 0.1, "medium": (value - 0.1) / 0.1, "hard": 0.0}
    elif 0.2 < value <= 0.3:
        return {"soft": 0.0, "medium": (0.3 - value) / 0.1, "hard": (value - 0.2) / 0.1}
    else:
        return {"soft": 0.0, "medium": 0.0, "hard": 1.0}

def fuzzify_compactness_mean(value):
    if value < 0.05:
        return {"low": 1.0, "medium": 0.0, "high": 0.0}
    elif 0.05 <= value <= 0.15:
        return {"low": (0.15 - value) / 0.1, "medium": (value - 0.05) / 0.1, "high": 0.0}
    elif 0.15 < value <= 0.3:
        return {"low": 0.0, "medium": (0.3 - value) / 0.15, "high": (value - 0.15) / 0.15}
    else:
        return {"low": 0.0, "medium": 0.0, "high": 1.0}

def fuzzify_concavity_mean(value):
    if value < 0.1:
        return {"low": 1.0, "medium": 0.0, "high": 0.0}
    elif 0.1 <= value <= 0.2:
        return {"low": (0.2 - value) / 0.1, "medium": (value - 0.1) / 0.1, "high": 0.0}
    elif 0.2 < value <= 0.4:
        return {"low": 0.0, "medium": (0.4 - value) / 0.2, "high": (value - 0.2) / 0.2}
    else:
        return {"low": 0.0, "medium": 0.0, "high": 1.0}

def fuzzify_concave_points_mean(value):
    if value < 0.05:
        return {"low": 1.0, "medium": 0.0, "high": 0.0}
    elif 0.05 <= value <= 0.15:
        return {"low": (0.15 - value) / 0.1, "medium": (value - 0.05) / 0.1, "high": 0.0}
    elif 0.15 < value <= 0.25:
        return {"low": 0.0, "medium": (0.25 - value) / 0.1, "high": (value - 0.15) / 0.1}
    else:
        return {"low": 0.0, "medium": 0.0, "high": 1.0}

def fuzzify_data_point(crisp_data_point):
    return {
        "radius_mean": fuzzify_radius_mean(crisp_data_point["radius_mean"]),
        "texture_mean": fuzzify_texture_mean(crisp_data_point["texture_mean"]),
        "perimeter_mean": fuzzify_perimeter_mean(crisp_data_point["perimeter_mean"]),
        "area_mean": fuzzify_area_mean(crisp_data_point["area_mean"]),
        "smoothness_mean": fuzzify_smoothness_mean(crisp_data_point["smoothness_mean"]),
        "compactness_mean": fuzzify_compactness_mean(crisp_data_point["compactness_mean"]),
        "concavity_mean": fuzzify_concavity_mean(crisp_data_point["concavity_mean"]),
        "concave_points_mean": fuzzify_concave_points_mean(crisp_data_point["concave_points_mean"]),
    }

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve, accuracy_score
import matplotlib.pyplot as plt

def fuzzify_dataset(data):
    return data.apply(lambda row: fuzzify_data_point(row), axis=1)

def convert_fuzzy_to_categories(fuzzy_data):
    result = {}
    for feature, memberships in fuzzy_data.items():
        max_category = max(memberships, key=memberships.get)
        result[feature] = max_category
    return result

def initialize_population(pop_size, fuzzy_info):
    population = []
    for _ in range(pop_size):
        rule_features = np.random.choice(list(fuzzy_info.keys()), size=4, replace=False)
        rule_conditions = [
            {
                "feature": feature,
                "fuzzy": np.random.choice([f["fuzzy"] for f in fuzzy_info[feature]]),
            }
            for feature in rule_features
        ]
        output_fuzzy = np.random.choice(["benign", "malignant"])
        population.append({"conditions": rule_conditions, "output": output_fuzzy})
    return population

def convert_to_readable_rules(rule):
    readable_conditions = " AND ".join([
        f"{condition['feature']} IS {condition['fuzzy']}"
        for condition in rule["conditions"]
    ])
    return f"IF {readable_conditions}, THEN OUTPUT IS {rule['output']}"

def evaluate_rule(rule, data):
    predictions = []
    for _, crisp_data_point in data.iterrows():
        fuzzy_data_point = fuzzify_data_point(crisp_data_point)
        match = True
        for condition in rule["conditions"]:
            feature = condition["feature"]
            fuzzy_label = condition["fuzzy"]
            if fuzzy_data_point[feature][fuzzy_label] == 0:
                match = False
                break
        predictions.append(rule["output"] if match else None)
    return predictions

def evaluate_population(population, data):
    final_predictions = []
    for _, crisp_data_point in data.iterrows():
        for rule in population:
            fuzzy_data_point = fuzzify_data_point(crisp_data_point)
            match = True
            for condition in rule["conditions"]:
                feature = condition["feature"]
                fuzzy_label = condition["fuzzy"]
                if fuzzy_data_point[feature][fuzzy_label] == 0:
                    match = False
                    break
            if match:
                final_predictions.append(1 if rule["output"] == "malignant" else 0)
                break
        else:
            final_predictions.append(0)
    return final_predictions

def crossover(parent1, parent2):
    point = np.random.randint(1, len(parent1["conditions"]))
    child1_conditions = parent1["conditions"][:point] + parent2["conditions"][point:]
    child2_conditions = parent2["conditions"][:point] + parent1["conditions"][point:]
    child1 = {"conditions": child1_conditions, "output": parent1["output"]}
    child2 = {"conditions": child2_conditions, "output": parent2["output"]}
    return child1, child2

def mutate(rule, fuzzy_info, mutation_rate):
    if np.random.rand() < mutation_rate:
        idx = np.random.randint(len(rule["conditions"]))
        feature = rule["conditions"][idx]["feature"]
        new_fuzzy = np.random.choice([f["fuzzy"] for f in fuzzy_info[feature]])
        rule["conditions"][idx]["fuzzy"] = new_fuzzy
    if np.random.rand() < mutation_rate:
        rule["output"] = "benign" if rule["output"] == "malignant" else "malignant"
    return rule

def genetic_algorithm(data, labels, pop_size=20, num_generations=100, mutation_rate=0.1):
    fuzzy_info = {
        "radius_mean": [{"fuzzy": "small"}, {"fuzzy": "medium"}, {"fuzzy": "large"}],
        "texture_mean": [{"fuzzy": "soft"}, {"fuzzy": "medium"}, {"fuzzy": "hard"}],
        "perimeter_mean": [{"fuzzy": "small"}, {"fuzzy": "medium"}, {"fuzzy": "large"}],
        "area_mean": [{"fuzzy": "small"}, {"fuzzy": "medium"}, {"fuzzy": "large"}],
        "smoothness_mean": [{"fuzzy": "soft"}, {"fuzzy": "medium"}, {"fuzzy": "hard"}],
        "compactness_mean": [{"fuzzy": "low"}, {"fuzzy": "medium"}, {"fuzzy": "high"}],
        "concavity_mean": [{"fuzzy": "low"}, {"fuzzy": "medium"}, {"fuzzy": "high"}],
        "concave_points_mean": [{"fuzzy": "low"}, {"fuzzy": "medium"}, {"fuzzy": "high"}],
    }
    population = initialize_population(pop_size, fuzzy_info)

    for generation in range(num_generations):
        print(f"\nGeneration {generation + 1}")
        for i, rule in enumerate(population):
            readable_rule = convert_to_readable_rules(rule)
            print(f"Rule {i + 1}: {readable_rule}")

        fitness_scores = evaluate_population(population, data)

        print(f"Best Fitness in Generation {generation + 1}: {max(fitness_scores)}")

        new_population = []
        for i in range(0, pop_size, 2):
            parent1, parent2 = population[i], population[i + 1]
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([mutate(child1, fuzzy_info, mutation_rate), mutate(child2, fuzzy_info, mutation_rate)])
        population = new_population

    return population

raw_data = pd.read_csv("data.csv")
labels = raw_data['diagnosis'].map({'B': 0, 'M': 1})
raw_data.rename(columns={'concave points_mean': 'concave_points_mean'}, inplace=True)

features = [
    'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
    'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave_points_mean'
]

final_population = genetic_algorithm(raw_data[features], labels)

print("\nFinal Population:")
for i, rule in enumerate(final_population):
    readable_rule = convert_to_readable_rules(rule)
    print(f"Rule {i + 1}: {readable_rule}")



# Evaluate final population on all data
final_predictions = evaluate_population(final_population, raw_data[features])

# Calculate Accuracy
accuracy = accuracy_score(labels, final_predictions)
print(f"Accuracy: {accuracy:.4f}")

# Mean Squared Error
mse = mean_squared_error(labels, final_predictions)
print(f"Mean Squared Error (MSE): {mse:.4f}")

# Root Mean Squared Error
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

# Area Under the Curve (AUC)
auc = roc_auc_score(labels, final_predictions)
print(f"Area Under the Curve (AUC): {auc:.4f}")





Generation 1
Rule 1: IF area_mean IS medium AND radius_mean IS large AND texture_mean IS soft AND concavity_mean IS high, THEN OUTPUT IS benign
Rule 2: IF radius_mean IS medium AND area_mean IS medium AND compactness_mean IS high AND perimeter_mean IS small, THEN OUTPUT IS benign
Rule 3: IF texture_mean IS hard AND area_mean IS medium AND perimeter_mean IS medium AND smoothness_mean IS soft, THEN OUTPUT IS malignant
Rule 4: IF concave_points_mean IS high AND concavity_mean IS low AND radius_mean IS medium AND compactness_mean IS high, THEN OUTPUT IS benign
Rule 5: IF perimeter_mean IS medium AND concavity_mean IS low AND radius_mean IS medium AND texture_mean IS medium, THEN OUTPUT IS benign
Rule 6: IF texture_mean IS soft AND compactness_mean IS high AND perimeter_mean IS small AND concavity_mean IS low, THEN OUTPUT IS benign
Rule 7: IF concave_points_mean IS low AND perimeter_mean IS small AND area_mean IS medium AND compactness_mean IS high, THEN OUTPUT IS malignant
Rule 8: IF area